# Meu primeiro programa com Qubits

O objetivo deste documento é demonstrar a execução de um pequeno programa baseado em Qubits. O programa será projetado na forma de um circuito com portas lógicas quânticas. 

Utilizaremos a linguagem Qiskit da IBM. Esta linguagem pode ser utilizada a partir de um programa Python. O nosso programa Python foi criado em um Jupyter Notebook no ambiente Google Colab.

Para poder utilizar a Qiskit no ambiente Google Colab precisamos instalar a mesma. A célula a seguir faz a instalação do pacote `qiskit` no ambiente do Google Colab.

In [ ]:
!pip install qiskit

Uma vez que o pacote esteja instalado podemos carrega-lo no nosso ambiente Python. O comando abaixo carrega (importa) o pacote `qiskit` para o ambiente Python e atribuiu ao mesmo o nome (apelido) `qk`

In [ ]:
import qiskit as qk

Nosso circuito será composto de dois Qubits, os quais serão inicializados no estado $|{0}\rangle$. 

O primeiro Qubit passará por um Hadamart gate (colocando-o em sobreposição dos estados $|{0}\rangle$ e $|{1}\rangle$. 

Em seguida é aplicado um CNOT gate ao conjunto o que os colocará em entrelaçamento e sobreposição, nos estados $|{00}\rangle$ e $|{11}\rangle$.

Por último medimos o estado de ambos os Qubits.

A forma gráfica do circuito é apresentada no último comando da célula abaixo.

In [ ]:
# Criamos os Qubits
q = qk.QuantumRegister(2)

# Criamos os Bits
c = qk.ClassicalRegister(2)

# Criamos o circuito (com Qubits e Bits)
circuit = qk.QuantumCircuit(q, c)

# Resetamos o primeiro Qubit para o estado |0>
circuit.reset(q[0])

# Resetamos o segundo Qubit para o estado |0>
circuit.reset(q[1])

# Aplicamos um Hadamart Gate no Qubit[0]
circuit.h(q[0])

# e um CNOT Gate na saida do Hadamart (Qubit[0], Qubit[1])
#  A primeira posição é do controle e a segunda do servo
circuit.cx(q[0],q[1])

# Medimos o estado do Qubit
circuit.measure(q, c)

# Visualizamos o circuito
print(circuit)

            ┌───┐     ┌─┐   
q14_0: ─|0>─┤ H ├──■──┤M├───
            └───┘┌─┴─┐└╥┘┌─┐
q14_1: ─|0>──────┤ X ├─╫─┤M├
                 └───┘ ║ └╥┘
 c1: 2/════════════════╩══╩═
                       0  1 


A computação quântica é essencialmente probabilística. Pense nela como uma simulação de Monte Carlo em planilha, onde ao invés do comando `aleatorio()` do Excel (em Português) você tem o circuito quântico. Sendo assim, cada vez que o circuito for executado, será gerada uma amostra dos possíveis resultados. Observe no entando que neste tipo de simulação, os resultados são sempre **discretos** isto é números inteiros que representam os estados que o circuito "decidiu" nos apresentar quando tentamos medi-lo.

Temos então que criar o equivalente à tabela de dados da simulação de Monte Carlo no Excel. Esta "tabela" é gerada enviando o circuito para um dos *backends* que a IBM disponibiliza. O método mais rápido de executar a simulação é através do simulador `BasicAer` o que nada mais é que um simulador de Qubits que roda na própria máquina local. Este tipo de simulação é pseudo-aleatória, tal como qualquer simulação baseada em algoritmos clássicos (baseados em bits). 

O `BasicAer` é o ambiente local que irá rodar a simulação. A simulação em si precisa de um ambiente `backend`. Pense que ela roda em modo batch. O `BasicAer` precisa enviar o circuito para um *motor* rodar a mesma. No caso deste exemplo o *motor* ou *backend* é o `qasm_simulator`. Sendo assim, primeiro instanciamos a classe `BasicAer`, criando um objeto chamado `simulator` o qual irá rodar a simulação no backend `qasm_simulator`. 

Com o circuito (objeto de nome `circuit`) e o simulator (objeto de nome `simulator`) criados (isto é com as respectivas classes instanciadas), pedimos para o `circuit` ser executado pelo `simulator` um determinado número de vezes (`shots`, neste caso 1.000) através do comando qiskit `execute`.

Neste caso estamos interessados nas contagens, para determinar que os Qubits entrelaçados mostraram apenas os estados $|{00}\rangle$ e $|{11}\rangle$ quando medidos. Sendo assim poderíamos ter obtido o dicionário python com os resultados fazendo diretamente: `job.result().get_counts(circuit)`

In [ ]:
# Instanciando o simulador Qiskit Aer's Qasm
# Observe que este simulador roda na sua máquina local
simulator = qk.BasicAer.get_backend('qasm_simulator')

# Executando a simulação (1000 medidas)
job = qk.execute(circuit, simulator, shots=1000)

# Capturando os resultados
result = job.result()

# Obtendo as contagens dos estados medidos
counts = result.get_counts(circuit)

# Apresentando os resultados
print(counts)

{'00': 475, '11': 525}


Vamos agora executar nosso circuito em uma máquina quântica real. A célula abaixo precisa ser executada apenas se a `qiskit` ainda não estiver instalada no Google Colab (o que precisa ser feito uma vez por sessão). Por isso deixei a célula a seguir comentada.

In [ ]:
#!pip install qiskit

In [ ]:
# Vamos usar um computador quântico real
import qiskit as qk

Primeiro devemos ir no site `https://quantum-computing.ibm.com`, nos registrar e obter a nossa API para código python. Ela é a "assinatura" para as máquinas da IBM que o "job" foi enviado por um usuário registrado. Por questões de privacidade eliminei o valor da chave deste Jupyter Notebook. Como a mesma já foi salva na área do meu ambiente Colab (através do comando `qk.IBMQ.load_account(chave)`), posso apenas carrega-la com `qk.IBMQ.load_account`. Por isso a célula abaixo está comentada. 

In [ ]:
# Obter a chave para uso da API IBM em : https://quantum-computing.ibm.com
# chave = 'A minha chave já foi salva neste ambiente uma vez'
# qk.IBMQ.save_account(chave)

configrc.store_credentials:WARNING:2021-05-28 19:51:33,376: Credentials already present. Set overwrite=True to overwrite.


In [ ]:
# Carregar a conta na sessão atual
qk.IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-05-28 21:37:16,733: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

Carregamos nosso usuário na célula anterior. Agora precisamos indicar um provedor de serviço. O provedor de serviço default é apresentado a seguir apenas por uma questão didática, pois ele já foi carregado pelo comando `qk.IBMQ.load_account()`. Para tornar nosso trabalho mais estruturado definimos este provedor como `provider_1`. O provedor precisa de um backend (uma máquina) tal como no caso anterior. Usaremos para o provedor default (`provider_1`) o backend do simulador da IBM `ibmq_qasm_simulator`. Observe que agora o simulador roda na nuvem.

In [ ]:
# O provedor default (público) é hub='ibm-q', group='open, project='main'
# O código abaixo é executado para conhecimento
provider_1 = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# No provedor público vamos utilizar o simulador de nuvem
backend_1 = provider_1.get_backend('ibmq_qasm_simulator')

Utilizarei também o provedor de serviços para instituições de ensino da IBM. Os dados podem ser vistos a seguir. Neste caso chamaremos este provedor de `provider_2`. Para o caso do `provider_2` vamos utilizar como backend uma máquina real, por exemplo a `ibmq_jakarta`. Outras máquinas poderiam ter sido utilizadas. Para uma lista completa, devemos olhar as máquinas disponíveis para a nossa conta em `https://quantum-computing.ibm.com`.

In [ ]:
# Utilizaremos também o provedor abaixo, o qual tem jobs ilimitados
provider_2 = qk.IBMQ.get_provider(hub='ibm-q-education', group='fgv-1', project='ml-business-app')

# Para este provedor utilizaremos a máquina ibmq_jakarta
backend_2 = provider_2.get_backend('ibmq_jakarta')

Neste segundo exemplo vamos criar um circuito um pouco distinto. Vamos gerar um número ao acaso entre 0 e 7. Para isso precisamos de 3 Qubits. Cada Qubit vai gerar um número 0 ou 1 ao acaso. Ao final combinamos os 3 números binários de 1 dígito em um número binário de 3 dígitos e convertemos o mesmo para base decimal. O circuito pode ser visto a seguir.

In [ ]:
# Com n Qubits pode-se gerar um número aleatório entre 0 e 2^n - 1
n = 3

q = qk.QuantumRegister(n)
c = qk.ClassicalRegister(n)
circ = qk.QuantumCircuit(q, c)

# Aplicando um Hadamart nos 3 Qubits
for i in range(n):
  circ.h(q[i])

# Efetuando a medida
circ.measure(q,c)

# Visualisando o circuito
print(circ) 

      ┌───┐┌─┐      
q0_0: ┤ H ├┤M├──────
      ├───┤└╥┘┌─┐   
q0_1: ┤ H ├─╫─┤M├───
      ├───┤ ║ └╥┘┌─┐
q0_2: ┤ H ├─╫──╫─┤M├
      └───┘ ║  ║ └╥┘
c0: 3/══════╩══╩══╩═
            0  1  2 


O comando a seguir envia o pedido de uma amostra (um "shot") para o backend escolhido (`ibmq_jakarta`) através do nosso provedor (`provider_2`). Se olharmos a nossa fila, na nossa conta, no site `https://quantum-computing.ibm.com` veremos que temos um *job* enviado para a máquina `ibmq_jakarta`. 

In [ ]:
# qk.execute envia para o backend. Conferindo no iqmq explorer aparece o job
new_job = qk.execute(circ, backend_2, shots=1)

Depois que o código for executado, o resultado é devolvido da mesma forma que na simulação local através de um dicionário python. Observe que no caso da simulação que gerou este arquivo, a máquina devolveu `110:1`, ou seja uma contagem (uma medição) e o resultado desta única medição foi o estado `110` do conjunto de três qubits. Observe que neste caso cada Qubit estava em sobreposição apenas consigo mesmo, ou seja os Qubits não estavam entrelaçados entre si.

In [ ]:
# este resultado fica armazenado local. Porém só estará disponível depois 
#  do job ter sido executado. Ele retorna um dicionário python.
new_job.result().get_counts()

{'110': 1}

A partir deste ponto são mostradas as etapas em linguagem python para converter a chave `110` deste dicionário python de resultados em um número decimal. A parte "quântica" do nosso programa Python terminou aqui.

In [ ]:
# Extraimos as chaves deste dicionário python
new_job.result().get_counts().keys()

dict_keys(['110'])

In [ ]:
# Convertemos as chaves em uma lista
list(new_job.result().get_counts().keys())

['110']

In [ ]:
# Obtemos o primeiro elemento da lista
list(new_job.result().get_counts().keys())[0]

'110'

In [ ]:
# Convertemos em um número inteiro decimal
int(list(new_job.result().get_counts().keys())[0])

110

In [ ]:
# Convertemos em um número inteiro binário e obtemos sua representação decimal
int(list(new_job.result().get_counts().keys())[0],2)

6

Obtivemos então como resultado o número 6.